# W_GL_BALANCE_F ETL Process
### Fact Table - Staging to Fact with Dimension Lookups

In [ ]:
%sql
-- 1) PARAMETERS
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID, 2625555 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLBALANCEFACT'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING WITH DIMENSION LOOKUPS
CREATE OR REPLACE TEMP VIEW stg_balance_with_dims AS
SELECT
  fs.*,
  COALESCE(la.ROW_WID, 0) AS LEDGER_WID,
  COALESCE(acc.ROW_WID, 0) AS GL_ACCOUNT_WID,
  COALESCE(per.ROW_WID, 0) AS PERIOD_WID,
  COALESCE(ptype.ROW_WID, 0) AS PERIOD_TYPE_WID
FROM workspace.oracle_edw.w_gl_balance_fs fs
LEFT JOIN workspace.oracle_edw.w_ledger_d la
  ON fs.LEDGER_ID = la.INTEGRATION_ID
  AND fs.DATASOURCE_NUM_ID = la.DATASOURCE_NUM_ID
  AND la.DELETE_FLG = 'N'
LEFT JOIN workspace.oracle_edw.w_gl_account_d acc
  ON fs.GL_ACCOUNT_ID = acc.INTEGRATION_ID
  AND fs.DATASOURCE_NUM_ID = acc.DATASOURCE_NUM_ID
  AND acc.DELETE_FLG = 'N'
LEFT JOIN workspace.oracle_edw.w_gl_period_d per
  ON fs.PERIOD_ID = per.INTEGRATION_ID
  AND fs.DATASOURCE_NUM_ID = per.DATASOURCE_NUM_ID
LEFT JOIN workspace.oracle_edw.w_gl_period_type_d ptype
  ON fs.PERIOD_TYPE = ptype.INTEGRATION_ID;

In [ ]:
%sql
-- 3) MERGE INTO FACT TABLE
MERGE INTO workspace.oracle_edw.w_gl_balance_f AS tgt
USING (
  SELECT s.*, p.ETL_PROC_WID, current_timestamp() AS CURRENT_TS
  FROM stg_balance_with_dims s
  CROSS JOIN etl_params p
) src
ON tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
WHEN MATCHED THEN
  UPDATE SET
    tgt.LEDGER_WID = src.LEDGER_WID,
    tgt.GL_ACCOUNT_WID = src.GL_ACCOUNT_WID,
    tgt.PERIOD_WID = src.PERIOD_WID,
    tgt.BALANCE_ACCT_AMT = src.BALANCE_ACCT_AMT,
    tgt.BALANCE_LOC_AMT = src.BALANCE_LOC_AMT,
    tgt.ACTIVITY_ACCT_AMT = src.ACTIVITY_ACCT_AMT,
    tgt.ACTIVITY_LOC_AMT = src.ACTIVITY_LOC_AMT,
    tgt.W_UPDATE_DT = src.CURRENT_TS,
    tgt.ETL_PROC_WID = src.ETL_PROC_WID
WHEN NOT MATCHED THEN
  INSERT (
    LEDGER_WID, GL_ACCOUNT_WID, PERIOD_WID, PERIOD_TYPE_WID,
    BALANCE_ACCT_AMT, BALANCE_LOC_AMT, ACTIVITY_ACCT_AMT, ACTIVITY_LOC_AMT,
    ACTUAL_FLAG, BALANCE_TYPE_INDICATOR, TRANSLATED_FLAG,
    DATASOURCE_NUM_ID, INTEGRATION_ID, TENANT_ID,
    W_INSERT_DT, W_UPDATE_DT, ETL_PROC_WID
  )
  VALUES (
    src.LEDGER_WID, src.GL_ACCOUNT_WID, src.PERIOD_WID, src.PERIOD_TYPE_WID,
    src.BALANCE_ACCT_AMT, src.BALANCE_LOC_AMT, src.ACTIVITY_ACCT_AMT, src.ACTIVITY_LOC_AMT,
    src.ACTUAL_FLAG, src.BALANCE_TYPE_INDICATOR, src.TRANSLATED_FLAG,
    src.DATASOURCE_NUM_ID, src.INTEGRATION_ID, src.TENANT_ID,
    src.CURRENT_TS, src.CURRENT_TS, src.ETL_PROC_WID
  );

In [ ]:
%sql
-- 4) UPDATE CONTROL (Standard pattern)
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLBALANCEFACT' AS package_name,
    'W_GL_BALANCE_F' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID,
    p.EXECUTION_ID AS load_plan_id, current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.etl_load_date = src.etl_load_date
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name,
  src.etl_usage_code, src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
DROP VIEW IF EXISTS stg_balance_with_dims;
DROP VIEW IF EXISTS etl_params;